In [3]:
import os
import cv2
import numpy as np

# Define paths
source_dir = 'Plant_leave_diseases_dataset_without_augmentation'  # Change this to your actual path
output_dir = 'processed_edges_dataset'
os.makedirs(output_dir, exist_ok=True)

# Set target size for CNN input
target_size = (224, 224)

# Loop through all image files in class subfolders
for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith(('.jpg', '.jpeg', '.png', '.tiff', '.tif')):
            # Build full source path
            src_path = os.path.join(root, file)

            # Derive class subfolder structure
            rel_path = os.path.relpath(src_path, source_dir)
            class_folder = os.path.dirname(rel_path)
            dest_folder = os.path.join(output_dir, class_folder)
            os.makedirs(dest_folder, exist_ok=True)

            try:
                # Load image
                image = cv2.imread(src_path)
                if image is None:
                    continue

                # Convert to grayscale
                gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

                # Apply Sobel edge detection
                sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
                sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
                sobel_edge = cv2.magnitude(sobel_x, sobel_y)

                # Resize and normalize (convert back to 8-bit format)
                edge_resized = cv2.resize(sobel_edge, target_size)
                edge_uint8 = (edge_resized / edge_resized.max() * 255).astype('uint8')

                # Save image as PNG
                filename = os.path.splitext(file)[0] + ".png"
                save_path = os.path.join(dest_folder, filename)
                cv2.imwrite(save_path, edge_uint8)

            except Exception as e:
                print(f"Failed to process {file}: {e}")


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Path to your edge-detected dataset
edge_dataset_path = 'processed_edges_dataset'  # Update if needed

# Data generator setup (rescale and validation split)
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

# Train loader
train_generator = datagen.flow_from_directory(
    edge_dataset_path,
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

# Validation loader
val_generator = datagen.flow_from_directory(
    edge_dataset_path,
    target_size=(224, 224),
    color_mode='grayscale',
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# Simple CNN architecture
def create_simple_cnn(input_shape=(224, 224, 1), num_classes=train_generator.num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Create and train model
model = create_simple_cnn()
model.summary()

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

# Save the model
model.save("simple_cnn_edge_model.h5")


Found 44369 images belonging to 39 classes.
Found 11077 images belonging to 39 classes.


C:\Users\aruno\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 222, 222, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 186624)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    11,944,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 39)             │         2,535 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,965,351 (45.64 MB)

 Trainable params: 11,965,351 (45.64 MB)

 Non-trainable params: 0 (0.00 B)

C:\Users\aruno\anaconda3\envs\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 799s 575ms/step - accuracy: 0.4269 - loss: 2.1437 - val_accuracy: 0.6787 - val_loss: 1.0861
Epoch 2/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 400s 289ms/step - accuracy: 0.7887 - loss: 0.6997 - val_accuracy: 0.7063 - val_loss: 0.9900
Epoch 3/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 596s 430ms/step - accuracy: 0.9265 - loss: 0.2412 - val_accuracy: 0.7076 - val_loss: 1.1446
Epoch 4/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 634s 457ms/step - accuracy: 0.9787 - loss: 0.0757 - val_accuracy: 0.6674 - val_loss: 1.5354
Epoch 5/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 632s 456ms/step - accuracy: 0.9831 - loss: 0.0544 - val_accuracy: 0.6926 - val_loss: 1.7337
Epoch 6/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 630s 454ms/step - accuracy: 0.9875 - loss: 0.0401 - val_accuracy: 0.6995 - val_loss: 1.7058
Epoch 7/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 407s 293ms/step - accuracy: 0.9889 - loss: 0.0330 - val_accuracy: 0.6893 - val_loss: 1.9012
Epoch 8/10
1387/1387 ━━━━━━━━━━━━━━━━━━━━ 439s 316ms/step - ac